In [ ]:
from radiomics.featureextractor import RadiomicsFeatureExtractor 
import SimpleITK as sitk 
import os 
import numpy as np 
from library_dicom.dicom_processor.tools.pyradiomic import *
import csv

In [ ]:
csv_path= '/media/deeplearning/Elements/AHL2011_NIFTI.csv'
csv_file = []
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_file = []
    for row in reader :
        csv_file.append(row)
        
del csv_file[0] #enlever première ligne

In [ ]:
dataset = []
for row in csv_file : 
    if 'pet0' in row : 
        dataset.append(row)

In [ ]:
print("Nombre de mask à traiter :", len(dataset))

In [ ]:
#rename path 

dataset[0]
for data in dataset : 
    mask = "/".join(data[-1].split('/')[-2:])
    path = os.path.join('/media/deeplearning/Elements' ,mask)

    pet = "/".join(data[-2].split('/')[-2:])
    pet_path = os.path.join('/media/deeplearning/Elements' ,pet)
    data[-1] = path
    data[-2] = pet_path

In [ ]:
#find date in json 
import json 
liste_json = []
list_dir = os.listdir('/media/deeplearning/Elements/AHL_JSON')
for file_ in list_dir : 
    liste_json.append(os.path.join('/media/deeplearning/Elements/AHL_JSON', file_))


for data in dataset : 
    for json_path in liste_json : 
        with open(json_path) as json_file : 
            reader = json.load(json_file)

            uid = reader['study']['StudyInstanceUID']

            if uid == data[2] : 

                data.append(reader['study']['StudyDate'])

                break

In [ ]:
for data in dataset : 
    year = data[-1][0:4]
    month = data[-1][4:6]
    day = data[-1][6:8]
    l = [year, month, day]
    date = "_".join(l)

    data.append(date)

In [ ]:
#check if every data has its date
cpt = 0
for data in dataset : 
    if len(data) != 9 : 
        cpt += 1

cpt

In [ ]:
distance_max = []
for nifti in dataset: 
    try : 
        print(dataset.index(nifti))
        liste_center = get_center_of_mass(nifti[-3], thresh = True, pet_path=nifti[-4])
        d_max = calcul_distance_max(liste_center)
        nifti.append(d_max)
    except Exception as err : 
        print(err)

In [ ]:
good = []
problem = []
for data in dataset : 
    if len(data) == 10 : 
        good.append(data)
    else : 
        problem.append(data)

In [ ]:
print(len(problem), "problem during extracting features")

In [ ]:
nifti_directory = '/home/deeplearning/AHL'
filename = 'AHL2011_dMax_v3.csv'

with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "DATE", "STUDY_UID", "D_MAX"])
    for row in good: 
        csv_writer.writerow([row[1], row[-2], row[2], row[-1]])